In [1]:
from pandas import read_sql
from sqlalchemy import create_engine
import geopandas as gpd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import pandas as pd
from datetime import datetime as dtdt
import datetime
import dateutil
import re

import pandas as pd
import numpy as np
from pandas import DataFrame
import shapely

import re
import os
import sys

import json

def strongInt(s):
    try:
        return int(s)
    except:
        return 0


In [2]:
with open('graphql.json', 'r') as f:
    graphql = f.read()
    graphqlJson = json.loads(graphql)
graphqlJson

{u'data': {u'findListings': {u'__typename': u'ListingAggregation',
   u'aggregateData': {u'__typename': u'AggregateData',
    u'genreCount': [{u'__typename': u'Counts',
      u'count': 151,
      u'text': u'play'},
     {u'__typename': u'Counts', u'count': 91, u'text': u'event'},
     {u'__typename': u'Counts', u'count': 86, u'text': u'musicals'},
     {u'__typename': u'Counts', u'count': 73, u'text': u'comedy'},
     {u'__typename': u'Counts', u'count': 46, u'text': u'drama'},
     {u'__typename': u'Counts', u'count': 32, u'text': u'children'},
     {u'__typename': u'Counts', u'count': 24, u'text': u'cabaret'},
     {u'__typename': u'Counts', u'count': 20, u'text': u'performance-art'},
     {u'__typename': u'Counts', u'count': 18, u'text': u'solo-performance'},
     {u'__typename': u'Counts', u'count': 17, u'text': u'interactive'}],
    u'hasDiscounts': True},
   u'listings': [{u'__typename': u'Listing',
     u'date_closing': None,
     u'date_opening': None,
     u'date_preview': Non

In [11]:
graphqlJson['data']['findListings'].keys()

[u'pagination', u'listings', u'aggregateData', u'__typename']

In [3]:
pd.DataFrame(graphqlJson['data']['findListings']['aggregateData']['genreCount'])

,__typename,count,text
0,Counts,151,play
1,Counts,91,event
2,Counts,86,musicals
3,Counts,73,comedy
4,Counts,46,drama
5,Counts,32,children
6,Counts,24,cabaret
7,Counts,20,performance-art
8,Counts,18,solo-performance
9,Counts,17,interactive


In [194]:
rawFrame = pd.DataFrame(graphqlJson['data']['findListings']['listings'])
rawFrame['genresText'] = rawFrame.genres.apply(lambda gs: '|'.join(g['text'] for g in gs))
rawFrame['genresUrl'] = rawFrame.genres.apply(lambda gs: '|'.join(g['url_compat'] for g in gs))
rawFrame['logoPath'] = rawFrame.logo.apply(lambda ls: ls['sources'][0]['path'] if not pd.isnull(ls) else None)
rawFrame['marketText'] = rawFrame.markets.apply(lambda ms: ''.join([m['text'] for m in ms if not m['text'] in ['New York', 'New York City']]))
rawFrame['marketUrl'] = rawFrame.markets.apply(lambda ms: ''.join([m['url_compat'] for m in ms if not m['text'] in ['New York', 'New York City']]))
rawFrame['offerUrl'] = rawFrame.offer.apply(lambda os: [o['url'] for o in os if o['type'] == 'full_price'][0])
rawFrame['offerPrice'] = rawFrame.offer.apply(lambda os: [o['price'] for o in os if o['type'] == 'full_price'][0])
rawFrame['venueReference'] = rawFrame.venue.apply(lambda v:v['reference'])
rawFrame['venueTitle'] = rawFrame.venue.apply(lambda v:v['title'])
rawFrame['venueUrl'] = rawFrame.venue.apply(lambda v:v['url'])
Listings = rawFrame.drop(columns=['__typename', 'genres', 'logo', 'markets', 'name', 'offer', 'venue'])

Listings['priceSplit'] = Listings.offerPrice.apply(lambda s: sorted([float(m[1:]) for m in re.findall(r'\$\d+(?:\.\d+)?', s)]))
Listings['minPrice'] = Listings.priceSplit.apply(lambda ps: ps[0] if len(ps) > 0 else None)
Listings['maxPrice'] = Listings.priceSplit.apply(lambda ps: ps[-1] if len(ps) > 0 else None)
Listings['plusPrice'] = Listings.offerPrice.str.endswith('+')
Listings = Listings.drop(columns=['priceSplit'])

Listings.date_opening = Listings.date_opening.apply(lambda d: d.split('T')[0] if not pd.isnull(d) else None)
Listings.date_closing = Listings.date_closing.apply(lambda d: d.split('T')[0] if not pd.isnull(d) else None)
Listings.date_preview = Listings.date_preview.apply(lambda d: d.split('T')[0] if not pd.isnull(d) else None)

Listings

,date_closing,date_opening,date_preview,id,reference,review,title,genresText,genresUrl,logoPath,marketText,marketUrl,offerUrl,offerPrice,venueReference,venueTitle,venueUrl,minPrice,maxPrice,plusPrice
0,None,None,None,1174,carolines-on-broadway_1174,NaN,Carolines on Broadway,Comedy|Stand-up/Sketch Comedy,comedy|stand-up,/dyn/graphics/theatermania/v7w300/carolines-on...,Off-Broadway,off-broadway,http://www.carolines.com/,$16.50 +,carolines-on-broadway_732,Caroline's on Broadway,,16.50,16.50,True
1,None,None,None,113621,repertorio-espanol_113621,NaN,Repertorio Español,Play,play,/dyn/graphics/theatermania/v7w300/repertorio-e...,Off-Off-Broadway,off-off-broadway,http://repertorio.nyc/#/,$17-$72,repertorio-espanol_834,Repertorio Español,,17.00,72.00,False
2,None,2009-09-01,None,159035,the-best-stand-up-comedy-show-in-the-world_159035,NaN,The Best Stand-Up Comedy Show in the World,Comedy|Concert/Event,comedy|event,/dyn/graphics/theatermania/v2w300/the-best-sta...,,,http://www.theworldnyc.com/,$15.00 with a 2-beverage minimum,broadway-comedy-club_6088,Broadway Comedy Club,broadwaycomedyclub.com,15.00,15.00,False
3,None,None,None,13454,comic-strip-live_13454,NaN,Comic Strip Live,Stand-up/Sketch Comedy,stand-up,/dyn/graphics/theatermania/v8w300/comic-strip-...,Off-Off-Broadway,off-off-broadway,https://www.eventbrite.com/o/comic-strip-live-...,$15-$30,comic-strip-live_2705,Comic Strip Live,www.comicstriplive.com,15.00,30.00,False
4,None,2013-05-03,None,199848,distorted-diznee_199848,NaN,Distorted Diznee,Cabaret,cabaret,/dyn/graphics/theatermania/v2w300/distorted-di...,,,http://web.ovationtix.com/trs/pr/911956,$20.00,the-laurie-beechman-theatre_718,The Laurie Beechman Theatre,westbankcafe.com,20.00,20.00,False
5,None,None,None,303645,trike_303645,NaN,Trike,Comedy|Stand-up/Sketch Comedy,comedy|stand-up,/dyn/graphics/theatermania/v4w300/trike-logo-3...,Off-Off-Broadway,off-off-broadway,http://www.magnettheater.com/shows/42948-Trike,$10,magnet-theater_5004,Magnet Theater,,10.00,10.00,False
6,None,2011-12-08,None,188292,broadway-ballyhoo-a-show-tune-hootenanny_188292,NaN,Broadway Ballyhoo: A Show Tune Hootenanny!,Cabaret,cabaret,/dyn/graphics/theatermania/v3w300/broadway-bal...,,,http://feinsteinsattheregency.com/performance....,$15.00 - $25.00,the-laurie-beechman-theatre_718,The Laurie Beechman Theatre,westbankcafe.com,15.00,25.00,False
7,None,None,None,200174,asssscat-3000_200174,NaN,ASSSSCAT 3000,Concert/Event,event,/dyn/graphics/theatermania/v1w300/asssscat-300...,,,http://www.newyork.ucbtheatre.com/shows/view/12,$10.00,upright-citizens-brigade-theatre_883,Upright Citizens Brigade Theatre,www.ucbtheatre.com,10.00,10.00,False
8,None,2017-04-30,None,318898,the-golden-dragon-acrobats_318898,NaN,The Golden Dragon Acrobats,Family/Kids|Concert/Event,children|event,/dyn/graphics/theatermania/v3w300/the-golden-d...,,,http://www.brooklyncenter.org/,$12.50 - $25,brooklyn-center-for-the-performing-arts_2758,Brooklyn Center for the Performing Arts,BrooklynCenterOnline.org,12.50,25.00,False
9,None,2015-10-24,None,311454,michael-feinstein_311454,NaN,Michael Feinstein,Concert/Event,event,/dyn/graphics/theatermania/v1w300/michael-fein...,,,https://tickets.brooklyncenter.org/orderticket...,$36 - $65,brooklyn-center-for-the-performing-arts_2758,Brooklyn Center for the Performing Arts,BrooklynCenterOnline.org,36.00,65.00,False


In [195]:
Listings.to_excel('Theatermania Listings.xlsx')

In [185]:
Listings['priceSplit'] = Listings.offerPrice.apply(lambda s: sorted([float(m[1:]) for m in re.findall(r'\$\d+(?:\.\d+)?', s)]))
Listings['minPrice'] = Listings.priceSplit.apply(lambda ps: ps[0] if len(ps) > 0 else None)
Listings['maxPrice'] = Listings.priceSplit.apply(lambda ps: ps[-1] if len(ps) > 0 else None)
Listings['plusPrice'] = Listings.offerPrice.str.endswith('+')

In [184]:
Listings['minPrice'] = Listings.priceSplit.apply(lambda ps: ps[0] if len(ps) > 0 else None)
Listings['maxPrice'] = Listings.priceSplit.apply(lambda ps: ps[-1] if len(ps) > 0 else None)
Listings['plusPrice'] = Listings.offerPrice.str.endswith('+')

In [174]:
Listings[Listings.priceSplit.apply(lambda x:len(x)<2)][['offerPrice', 'priceSplit']]

,offerPrice,priceSplit
0,$16.50 +,[16.50]
2,$15.00 with a 2-beverage minimum,[15.00]
4,$20.00,[20.00]
5,$10,[10]
7,$10.00,[10.00]
10,$29,[29]
12,,[]
14,,[]
16,,[]
17,,[]


In [166]:
rawFrame['marketText'] = rawFrame.markets.apply(lambda ms: ''.join([m['text'] for m in ms if not m['text'] in ['New York', 'New York City']]))
rawFrame['marketUrl'] = rawFrame.markets.apply(lambda ms: ''.join([m['url_compat'] for m in ms if not m['text'] in ['New York', 'New York City']]))

In [111]:
rawFrame.venue.apply(lambda os: os.keys() if len(os)>0 else None)[0]

[u'reference', u'title', u'url', u'__typename', u'markets', u'id']

In [104]:
rawFrame['offerUrl'] = rawFrame.offer.apply(lambda os: [o['url'] for o in os if o['type'] == 'full_price'])
rawFrame['offerPrice'] = rawFrame.offer.apply(lambda os: [o['price'] for o in os if o['type'] == 'full_price'])